In [1]:
# Solving the deterministc Optimal Growth model by Policy Function Interation 
# adapted from Fabrice Collard's Matlab code, http://fabcol.free.fr/
# tested in Julia 1.5.0
# this code is part of chapter 4, "Dynamic Programming" from the book: "Introduction to Quantitative Macroeconomics with Julia"
# Academic Press - Elsevier
# for comments, email at: petre(dot)caraiani(at)gmail(dot)com

#using Plots
using LinearAlgebra;
using Plots;

sigma   = 1.50;                     # utility parameter
delta   = 0.10;                     # depreciation rate
beta    = 0.95;                     # discount factor
alpha   = 0.30;                     # capital elasticity of output

nbk     = 1000;                     # number of data points in the grid
crit    = 1;                        # convergence criterion
iter    = 1;                        # iteration
tol     = 1e-6;                     # convergence parameter

#steady state capital stock
ks      = ((1-beta*(1-delta))/(alpha*beta))^(1/(alpha-1));
dev     = 0.9;                      # maximal deviation from steady state
kmin    = (1-dev)*ks;               # lower bound on the grid
kmax    = (1+dev)*ks;               # upper bound on the grid
devk    = (kmax-kmin)/(nbk-1);      # implied increment
kgrid   = collect(LinRange(kmin,kmax,nbk)); # builds the grid
#kgrid   = kgrid';
v       = zeros(nbk,1);             # value function
c       = zeros(nbk,1);             # consumption

kp0     = kgrid;                    # initial guess on k(t+1)
dr      = zeros(nbk,1);             # decision rule (will contain indices)
kp      = zeros(nbk,1);



In [2]:
while crit>tol

for i=1:nbk
    
    imin    = max(ceil(((1-delta)*kgrid[i]-kmin)/devk)+1,1);
    imax    = min(floor((kgrid[i]^alpha+(1-delta)*kgrid[i]-kmin)/devk)+1,nbk);
    imin=trunc(Int, imin);
    imax=trunc(Int, imax);   
   
    c_temp       = kgrid[i]^alpha+(1-delta)*kgrid[i].-kgrid[imin:imax];
    util_temp    = (c_temp.^(1-sigma).-1)./(1-sigma);
    (v1,idr)= findmax(util_temp+beta*v[imin:imax]);
    dr[i]   = imin-1+idr;
    dr[i]   = trunc(Int, dr[i]);

  end;
    
# decision rules
kp   = zeros(nbk,1);
for i=1:nbk
   index=trunc(Int, dr[i]);

   kp[i]= kgrid[index];
  
end;


    c   = kgrid.^alpha+(1-delta).*kgrid-kp;

    util= (c.^(1-sigma).-1)./(1-sigma);

    Q   = zeros(nbk,nbk);

    for i=1:nbk;
       index=trunc(Int, dr[i]);

        Q[i,index] = 1;
    end

    A=(Matrix{Float64}(I, nbk, nbk)-beta*Q);    
    B=util;

    Tv  = \(A, B);
    crit= maximum(abs.(kp-kp0));
    
    v   = copy(Tv);
    kp0 = copy(kp);

    iter= iter+1;

end;


In [3]:
#Pkg.add("Plots")
#using Plots
plotly() # Choose the Plotly.jl backend for web interactivity
#plot(k',c',linewidth=1,label="Consumption",title="Consumption vs capital stock")
plot(kp,c,linewidth=1,label="Consumption",title="Consumption vs Capital stock")

┌ Info: For saving to png with the Plotly backend ORCA has to be installed.
└ @ Plots C:\Users\PETRE\.julia\packages\Plots\5srrj\src\backends.jl:371


<!DOCTYPE html>
 
 
 Plots.jl